# Baseline Experiment: LightGBM with Engineered Features

This notebook implements a baseline model using LightGBM on engineered features from the Reddit pizza requests dataset.

## Strategy
- Focus on the highly predictive user flair feature ('shroom' and 'PIF' have 100% success rate)
- Engineer basic text features (length, word count, etc.)
- Use metadata features (account age, activity metrics)
- Apply stratified k-fold cross-validation
- Handle class imbalance with appropriate techniques

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Load and Explore Data

In [ ]:
# Load training data
train_data = []
with open('/home/data/train.json', 'r') as f:
    lines = f.readlines()
    print(f"Total lines in file: {len(lines)}")
    
    for i, line in enumerate(lines):
        line = line.strip()
        if line:  # Skip empty lines
            try:
                train_data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error parsing line {i}: {e}")
                print(f"Line content: {line[:100]}...")
                continue

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")
print(f"\nTarget distribution:")
print(train_df['requester_received_pizza'].value_counts(normalize=True))

In [ ]:
# Load test data
test_data = []
with open('/home/data/test.json', 'r') as f:
    for line in f:
        if line.strip():  # Skip empty lines
            test_data.append(json.loads(line))

test_df = pd.DataFrame(test_data)
print(f"Test data shape: {test_df.shape}")
print(f"Columns: {test_df.columns.tolist()}")

## Feature Engineering

In [ ]:
def engineer_features(df):
    """Engineer features from the raw data"""
    features = pd.DataFrame()
    
    # Critical feature: User flair (highly predictive according to EDA)
    features['user_flair'] = df['requester_user_flair'].fillna('None')
    features['has_flair'] = (features['user_flair'] != 'None').astype(int)
    features['is_shroom'] = (features['user_flair'] == 'shroom').astype(int)
    features['is_pif'] = (features['user_flair'] == 'PIF').astype(int)
    
    # Text features
    # Use request_text_edit_aware to avoid leakage from edits
    text_col = 'request_text_edit_aware' if 'request_text_edit_aware' in df.columns else 'request_text'
    
    # Combine title and text for unified analysis
    df['combined_text'] = df['request_title'].fillna('') + ' ' + df[text_col].fillna('')
    
    # Basic text length features
    features['text_length'] = df['combined_text'].str.len()
    features['word_count'] = df['combined_text'].str.split().str.len()
    features['title_length'] = df['request_title'].fillna('').str.len()
    
    # Account age features (log transform for heavy-tailed distributions)
    features['account_age_days'] = df['requester_account_age_in_days_at_request']
    features['log_account_age'] = np.log1p(features['account_age_days'])
    
    # Activity metrics
    features['total_comments'] = df['requester_number_of_comments_at_request']
    features['total_posts'] = df['requester_number_of_posts_at_request']
    features['comments_per_post'] = features['total_comments'] / (features['total_posts'] + 1)
    features['log_comments'] = np.log1p(features['total_comments'])
    features['log_posts'] = np.log1p(features['total_posts'])
    
    # RAOP-specific activity
    features['raop_comments'] = df['requester_number_of_comments_in_raop_at_request']
    features['raop_posts'] = df['requester_number_of_posts_on_raop_at_request']
    features['log_raop_comments'] = np.log1p(features['raop_comments'])
    features['log_raop_posts'] = np.log1p(features['raop_posts'])
    
    # Vote-related features
    features['upvotes_minus_downvotes'] = df['requester_upvotes_minus_downvotes_at_request']
    features['upvotes_plus_downvotes'] = df['requester_upvotes_plus_downvotes_at_request']
    features['log_upvotes'] = np.log1p(features['upvotes_plus_downvotes'])
    
    # Subreddit diversity
    features['num_subreddits'] = df['requester_number_of_subreddits_at_request']
    features['log_num_subreddits'] = np.log1p(features['num_subreddits'])
    
    # Request-specific features
    features['request_comments'] = df['request_number_of_comments_at_retrieval']
    features['request_upvotes'] = df['number_of_upvotes_of_request_at_retrieval']
    features['request_downvotes'] = df['number_of_downvotes_of_request_at_retrieval']
    features['request_net_votes'] = features['request_upvotes'] - features['request_downvotes']
    
    # Time-based features
    features['unix_timestamp'] = df['unix_timestamp_of_request']
    features['hour_of_day'] = pd.to_datetime(features['unix_timestamp'], unit='s').dt.hour
    features['day_of_week'] = pd.to_datetime(features['unix_timestamp'], unit='s').dt.dayofweek
    features['is_weekend'] = features['day_of_week'].isin([5, 6]).astype(int)
    
    # Post was edited (potential leakage indicator)
    features['post_was_edited'] = df['post_was_edited'].astype(int)
    
    return features

# Engineer features for train and test
train_features = engineer_features(train_df)
test_features = engineer_features(test_df)

print(f"Engineered features shape: {train_features.shape}")
print(f"Feature columns: {train_features.columns.tolist()}")

In [ ]:
# Handle categorical features
# User flair is critical - let's use one-hot encoding
flair_dummies = pd.get_dummies(train_features['user_flair'], prefix='flair')
train_features = pd.concat([train_features, flair_dummies], axis=1)

# Apply same encoding to test
test_flair_dummies = pd.get_dummies(test_features['user_flair'], prefix='flair')
# Ensure same columns in both train and test
for col in flair_dummies.columns:
    if col not in test_flair_dummies.columns:
        test_flair_dummies[col] = 0
test_flair_dummies = test_flair_dummies[flair_dummies.columns]

test_features = pd.concat([test_features, test_flair_dummies], axis=1)

# Drop the original categorical column
train_features = train_features.drop('user_flair', axis=1)
test_features = test_features.drop('user_flair', axis=1)

print(f"Final feature shape: {train_features.shape}")
print(f"Sample features:\n{train_features.head()}")

## TF-IDF Features for Text

In [ ]:
# Create TF-IDF features for the text
# Use a subset of features to avoid dimensionality explosion
vectorizer = TfidfVectorizer(
    max_features=1000,  # Limit features for baseline
    stop_words='english',
    ngram_range=(1, 2),  # Unigrams and bigrams
    min_df=5,  # Ignore very rare terms
    max_df=0.95  # Ignore very common terms
)

# Fit on training text
train_text = train_df['combined_text'].fillna('')
tfidf_train = vectorizer.fit_transform(train_text)

# Transform test text
test_text = test_df['combined_text'].fillna('')
tfidf_test = vectorizer.transform(test_text)

print(f"TF-IDF features shape: {tfidf_train.shape}")

## Combine Features

In [ ]:
# Convert engineered features to sparse matrix format
from scipy.sparse import csr_matrix

train_engineered = csr_matrix(train_features.fillna(0).values)
test_engineered = csr_matrix(test_features.fillna(0).values)

# Combine TF-IDF and engineered features
X_train = hstack([tfidf_train, train_engineered])
X_test = hstack([tfidf_test, test_engineered])

y_train = train_df['requester_received_pizza'].values

print(f"Final training matrix shape: {X_train.shape}")
print(f"Final test matrix shape: {X_test.shape}")

## Model Training with Cross-Validation

In [ ]:
# Set up stratified k-fold cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize arrays for out-of-fold predictions
oof_predictions = np.zeros(len(train_df))
test_predictions = np.zeros(len(test_df))

# Track scores across folds
cv_scores = []

print(f"Training LightGBM model with {n_splits}-fold stratified CV...")

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"\nFold {fold + 1}/{n_splits}")
    
    # Split data
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(X_tr, label=y_tr)
    val_set = lgb.Dataset(X_val, label=y_val)
    
    # Define parameters
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_threads': 4
    }
    
    # Handle class imbalance with scale_pos_weight
    pos_rate = y_tr.mean()
    params['scale_pos_weight'] = (1 - pos_rate) / pos_rate
    
    # Train model
    model = lgb.train(
        params,
        train_set,
        num_boost_round=1000,
        valid_sets=[val_set],
        valid_names=['val'],
        callbacks=[
            lgb.early_stopping(50),
            lgb.log_evaluation(100)
        ]
    )
    
    # Make predictions
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    # Store predictions
    oof_predictions[val_idx] = val_pred
    test_predictions += test_pred / n_splits
    
    # Calculate fold score
    fold_score = roc_auc_score(y_val, val_pred)
    cv_scores.append(fold_score)
    
    print(f"Fold {fold + 1} AUC: {fold_score:.4f}")

# Calculate overall CV score
overall_score = roc_auc_score(y_train, oof_predictions)
print(f"\nOverall CV AUC: {overall_score:.4f}")
print(f"Mean CV AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

## Feature Importance Analysis

In [ ]:
# Get feature importance from the last fold model
feature_names = (list(vectorizer.get_feature_names_out()) + 
                 list(train_features.columns))

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("Top 20 most important features:")
print(importance_df.head(20))

## Create Submission

In [ ]:
# Create submission dataframe
submission = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_predictions
})

# Ensure the format matches sample submission
submission['requester_received_pizza'] = submission['requester_received_pizza'].clip(0, 1)

print("Submission preview:")
print(submission.head())
print(f"\nSubmission shape: {submission.shape}")
print(f"Prediction distribution:")
print(submission['requester_received_pizza'].describe())

# Save submission
submission.to_csv('/home/submission/submission.csv', index=False)
print("\nSubmission saved to /home/submission/submission.csv")